# Setup

In [0]:
#@title Enter Kaggle Credentials
import os
from getpass import getpass
os.environ['KAGGLE_USERNAME'] = 'ranik40' #@param {type:"string"}
print('Enter your kaggle key')
os.environ['KAGGLE_KEY'] = getpass()

Enter your kaggle key


In [26]:
!pip uninstall -y kaggle
!pip install -qq -U "kaggle" "tensorflow-model-optimization" "gcsfs" "focal-loss"

     |████████████████████████████████| 61kB 2.1MB/s 
     |████████████████████████████████| 174kB 8.0MB/s 
     |████████████████████████████████| 296kB 10.1MB/s 


In [0]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [2]:
TPU_NAME = 'grpc://' + os.environ['COLAB_TPU_ADDR']
TPU_NAME

'grpc://10.49.17.50:8470'

In [0]:
import sys

if 'google.colab' in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

# tensorflow/models Image Classification

In [113]:
!rm -rf models
!git clone https://github.com/tensorflow/models.git -b v2.2.0

Cloning into 'models'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 36784 (delta 23), reused 22 (delta 0), pack-reused 36738
Receiving objects: 100% (36784/36784), 520.38 MiB | 47.42 MiB/s, done.
Resolving deltas: 100% (24519/24519), done.
Note: checking out '93490036e00f37ecbe6693b9ff4ae488bb8e9270'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>



# EDA

In [31]:
# Get train & validation 
!gsutil cp gs://recursion-kaggle/melanoma/stratified_ex/fold0/idx_train.csv .
!gsutil cp gs://recursion-kaggle/melanoma/stratified_ex/fold0/idx_validation.csv .

Copying gs://recursion-kaggle/melanoma/stratified_ex/fold0/idx_train.csv...
/ [1 files][  1.5 MiB/  1.5 MiB]                                                
Operation completed over 1 objects/1.5 MiB.                                      
Copying gs://recursion-kaggle/melanoma/stratified_ex/fold0/idx_validation.csv...
/ [1 files][392.7 KiB/392.7 KiB]                                                
Operation completed over 1 objects/392.7 KiB.                                    


## Compute initial bias

In [32]:
import numpy as np
import pandas as pd

train = pd.read_csv('idx_train.csv')
train.head()

,image_name,patient_id,target,source,sex,age_approx,anatom_site_general_challenge,stratify_group,fold
0,ISIC_2637011,0,0,0,0,45,0,32,2
1,ISIC_0015719,1,0,0,1,45,1,28,1
2,ISIC_0052212,2,0,0,1,50,2,8,1
3,ISIC_0068279,3,0,0,1,45,0,2,4
4,ISIC_0074268,4,0,0,1,55,1,28,3


In [33]:
valid = pd.read_csv('idx_validation.csv')
valid.head()

,image_name,patient_id,target,source,sex,age_approx,anatom_site_general_challenge,stratify_group,fold
0,ISIC_0075914,8,0,0,0,30,3,50,0
1,ISIC_0076742,11,0,0,0,75,1,58,0
2,ISIC_0079038,16,0,0,0,70,3,50,0
3,ISIC_0080512,17,0,0,0,75,3,50,0
4,ISIC_0083035,24,0,0,0,50,3,50,0


In [34]:
train.shape[0], valid.shape[0]

(46648, 11809)

In [35]:
neg = train[train['target'] == 0].shape[0]
pos = train[train['target'] == 1].shape[0]

initial_bias = np.log([pos/neg])
initial_bias

array([-2.3498501])

# Pipeline

In [0]:
import sys
sys.path.append("models")

## Dataset

In [0]:
#@title Dataset
import os
# import numpy as np
from typing import Any, List, Optional, Tuple, Mapping, Union
from absl import logging
from dataclasses import dataclass
import tensorflow as tf
import tensorflow_datasets as tfds


from official.modeling.hyperparams import base_config
from official.vision.image_classification import augment
from official.vision.image_classification import preprocessing
from official.vision.image_classification import dataset_factory


class DatasetBuilder(dataset_factory.DatasetBuilder):
  def load_records(self) -> tf.data.Dataset:
    """Return a dataset loading files with TFRecords."""
    logging.info('Using TFRecords to load data.')

    if self.config.filenames is None:
      if self.config.data_dir is None:
        raise ValueError('Dataset must specify a path for the data files.')

      file_pattern = os.path.join(self.config.data_dir,
                                  '{}*'.format(self.config.split))
      
      if self.config.split in ['train', 'validation']:
        shuffle = True
      else:
        shuffle = False

      dataset = tf.data.Dataset.list_files(file_pattern, shuffle=shuffle)
    else:
      dataset = tf.data.Dataset.from_tensor_slices(self.config.filenames)
      if self.is_training:
        # Shuffle the input files.
        dataset.shuffle(buffer_size=self.config.file_shuffle_buffer_size)

    return dataset

  def pipeline(self,
               dataset: tf.data.Dataset,
               input_context: tf.distribute.InputContext = None
              ) -> tf.data.Dataset:
    """Build a pipeline fetching, shuffling, and preprocessing the dataset."""
    if input_context and input_context.num_input_pipelines > 1:
      dataset = dataset.shard(input_context.num_input_pipelines,
                              input_context.input_pipeline_id)

    if self.is_training and not self.config.cache:
      dataset = dataset.repeat()

    if self.config.builder == 'records':
      # Read the data from disk in parallel
      buffer_size = 8 * 1024 * 1024  # Use 8 MiB per file
      dataset = dataset.interleave(
          lambda name: tf.data.TFRecordDataset(name, buffer_size=buffer_size),
          cycle_length=16,
          num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    dataset = dataset.prefetch(self.global_batch_size)

    if self.config.cache:
      dataset = dataset.cache()

    if self.is_training:
      dataset = dataset.shuffle(self.config.shuffle_buffer_size)
      dataset = dataset.repeat()

    # Parse, pre-process, and batch the data in parallel
    if self.config.builder == 'records':
      if self.config.split in ['train', 'validation']:
        preprocess = self.parse_record
      else:
        preprocess = self.parse_test_record
    else:
      preprocess = self.preprocess

    dataset = dataset.map(preprocess,
                          num_parallel_calls=tf.data.experimental.AUTOTUNE)

    # Dataset balancing utilities
    @tf.function
    def class_func(image, label):
      return label

    @tf.function
    def drop_extra_label(extra_label, image_and_label):
      return image_and_label

    # if self.is_training:
    #   # Balance the dataset
    #   TARGET_DIST = [0.5, 0.5]
    #   INITIAL_DIST = [0.95, 0.05]

    #   resampler = tf.data.experimental.rejection_resample(
    #       class_func, 
    #       target_dist=TARGET_DIST,
    #       # seed=42,
    #       initial_dist=INITIAL_DIST
    #   )
    #   dataset = dataset.apply(resampler)

    dataset = dataset.batch(self.batch_size, drop_remainder=self.is_training)
    
    # if self.is_training:
    #   # The resampler returns creates (class, example) pairs from the output of the class_func. 
    #   # In this case, the example was already a (feature, label) pair, 
    #   # so use map to drop the extra copy of the labels
    #   dataset = dataset.map(
    #       drop_extra_label,
    #       num_parallel_calls=tf.data.experimental.AUTOTUNE
    #   )

    if self.config.split in ['test']:
      options = tf.data.Options()
      options.experimental_optimization.parallel_batch = True
      options.experimental_optimization.map_fusion = True
      # Note: Disabled map vectorization for balanced sampling
      # options.experimental_optimization.map_vectorization.enabled = True
      options.experimental_optimization.map_parallelization = True
      dataset = dataset.with_options(options)
      
    elif self.is_training and self.config.deterministic_train is not None:
      options = tf.data.Options()
      options.experimental_deterministic = self.config.deterministic_train
      options.experimental_slack = self.config.use_slack
      options.experimental_optimization.parallel_batch = True
      options.experimental_optimization.map_fusion = True
      # Note: Disabled map vectorization for balanced sampling
      # options.experimental_optimization.map_vectorization.enabled = True
      options.experimental_optimization.map_parallelization = True
      dataset = dataset.with_options(options)

    # Prefetch overlaps in-feed with training
    # Note: autotune here is not recommended, as this can lead to memory leaks.
    # Instead, use a constant prefetch size like the the number of devices.
    dataset = dataset.prefetch(self.config.num_devices)

    return dataset

  @tf.function
  def parse_record(self, record: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
    """Parse an ImageNet record from a serialized string Tensor."""
    keys_to_features = {
        'image':
            tf.io.FixedLenFeature((), tf.string, ''),
        # "age_approx": tf.io.FixedLenFeature([], tf.int64, -1),  
        # "sex": tf.io.FixedLenFeature([], tf.int64, -1),  
        'target':
            tf.io.FixedLenFeature([], tf.int64, -1)
    }
    
    parsed = tf.io.parse_single_example(record, keys_to_features)

    # age = tf.cast(parsed['age_approx'], tf.float32) / 30.
    # sex = tf.cast(parsed['sex'], tf.float32)

    # label = tf.reshape(parsed['target'], shape=[1])
    label = parsed['target']
    label = tf.cast(label, dtype=tf.int32)

    # image_bytes = tf.reshape(parsed['image'], shape=[])
    image_bytes = parsed['image']
    image, label = self.preprocess(image_bytes, label)

    return image, label
    # return (image, tf.stack([age,sex])), label

  def parse_test_record(self, record: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
    """Parse an ImageNet record from a serialized string Tensor."""
    keys_to_features = {
        'image':
            tf.io.FixedLenFeature((), tf.string, ''),
        # "age_approx": tf.io.FixedLenFeature([], tf.int64, -1),  
        # "sex": tf.io.FixedLenFeature([], tf.int64, -1),  
        'target':
            tf.io.FixedLenFeature([], tf.int64, -1),
        "image_name": 
            tf.io.FixedLenFeature((), tf.string)
    }
    
    parsed = tf.io.parse_single_example(record, keys_to_features)

    # age = tf.cast(parsed['age_approx'], tf.float32) / 30.
    # sex = tf.cast(parsed['sex'], tf.float32)

    # label = tf.reshape(parsed['target'], shape=[1])
    label = parsed['target']
    label = tf.cast(label, dtype=tf.int32)

    # image_bytes = tf.reshape(parsed['image'], shape=[])
    image_bytes = parsed['image']
    image, _ = self.preprocess(image_bytes, label)

    image_name = parsed['image_name']

    return image, image_name
    # return (image, tf.stack([age,sex])), image_name

dataset_factory.DatasetBuilder = DatasetBuilder

In [0]:
#@title EfficientNet Model
import math
import os
from typing import Any, Dict, Optional, Text, Tuple

from absl import logging
import tensorflow as tf

from official.vision.image_classification.efficientnet import efficientnet_model

class ModelConfig(efficientnet_model.ModelConfig):
  num_classes: int = 2


class EfficientNet(tf.keras.Model):
  """Wrapper class for an EfficientNet Keras model.
  Contains helper methods to build, manage, and save metadata about the model.
  """
  def __init__(self,
               config: ModelConfig = None,
               overrides: Dict[Text, Any] = None):
    """Create an EfficientNet model.
    Args:
      config: (optional) the main model parameters to create the model
      overrides: (optional) a dict containing keys that can override
                 config
    """
    overrides = overrides or {}
    config = config or ModelConfig()

    self.config = config.replace(**overrides)

    input_channels = self.config.input_channels
    model_name = self.config.model_name
    input_shape = (None, None, input_channels)  # Should handle any size image

    image_input = tf.keras.layers.Input(shape=input_shape)

    output = efficientnet_model.efficientnet(image_input, self.config)

    # Cast to float32 in case we have a different model dtype
    # output = tf.cast(output, tf.float32)

    logging.info('Building model %s with params %s',
                 model_name,
                 self.config)

    super(EfficientNet, self).__init__(
        inputs=image_input, 
        outputs=output, 
        name=model_name)

  @classmethod
  def from_name(cls,
                model_name: Text,
                model_weights_path: Text = None,
                copy_to_local: bool = False,
                overrides: Dict[Text, Any] = None):
    """Construct an EfficientNet model from a predefined model name"""
    model_configs = dict(efficientnet_model.MODEL_CONFIGS)
    overrides = dict(overrides) if overrides else {}

    # One can define their own custom models if necessary
    model_configs.update(overrides.pop('model_config', {}))

    if model_name not in model_configs:
      raise ValueError('Unknown model name {}'.format(model_name))

    config = model_configs[model_name]

    model = cls(config=config, overrides=overrides)

    # Pop the classification layer
    model = tf.keras.Model(model.inputs, model.layers[-3].output)

    if model_weights_path:
      if copy_to_local:
        tmp_file = os.path.join('/tmp', model_name + '.h5')
        model_weights_file = os.path.join(model_weights_path, 'model.h5')
        tf.io.gfile.copy(model_weights_file, tmp_file, overwrite=True)
        model_weights_path = tmp_file

      loaded_model = tf.keras.models.load_model(model_weights_path, compile=False)
      model.set_weights(loaded_model.get_weights())
    
    initial_bias = -2.3498501
    output_bias = tf.keras.initializers.Constant(initial_bias)

    output = model.output

    x = tf.keras.layers.Dense(
      # config.num_classes,
      1,
      kernel_initializer=efficientnet_model.DENSE_KERNEL_INITIALIZER,
      bias_initializer=output_bias,
      # kernel_regularizer=tf.keras.regularizers.l2(config.weight_decay),
      # bias_regularizer=tf.keras.regularizers.l2(config.weight_decay),
      name='logits')(output)

    x = tf.keras.layers.Activation('sigmoid', name='probs')(x)

    # Cast to float32 in case we have a different model dtype
    x = tf.cast(x, tf.float32)
    model = tf.keras.Model(inputs=model.inputs, outputs=x)
    return model

efficientnet_model.EfficientNet = EfficientNet
efficientnet_model.ModelConfig = ModelConfig


In [0]:
#@title Callbacks
from __future__ import absolute_import
from __future__ import division
# from __future__ import google_type_annotations
from __future__ import print_function

import os
from absl import logging

import tensorflow as tf
import tensorflow.keras.backend as K
from typing import Any, List, MutableMapping
from sklearn.metrics import roc_auc_score

from official.utils.misc import keras_utils
from official.vision.image_classification import callbacks


def get_callbacks(validation_dataset,
                  num_validation_samples,
                  model_checkpoint: bool = True,
                  include_tensorboard: bool = True,
                  time_history: bool = True,
                  track_lr: bool = True,
                  write_model_weights: bool = True,
                  initial_step: int = 0,
                  batch_size: int = 0,
                  log_steps: int = 0,
                  model_dir: str = None) -> List[tf.keras.callbacks.Callback]:
  """Get all callbacks."""
  model_dir = model_dir or ''
  callbacks = []

  if model_checkpoint:
    # ckpt_full_path = os.path.join(model_dir, 'model.ckpt-{epoch:04d}')
    ckpt_full_path = os.path.join(model_dir, 'model.ckpt')
    callbacks.append(
        ModelCheckpoint(
            validation_dataset,
            num_validation_samples,
            ckpt_full_path, 
            monitor='val_auc',
            mode='max',
            # save_freq=250,
            save_best_only=True,
            save_weights_only=True, verbose=1))
  if include_tensorboard:
    callbacks.append(
        CustomTensorBoard(
            log_dir=model_dir,
            track_lr=track_lr,
            initial_step=initial_step,
            write_images=write_model_weights))
    
  # reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
  #     monitor='val_auc', factor=0.8, patience=1, verbose=1, mode='max',
  #     min_delta=0.0001, cooldown=0, min_lr=1e-8, **kwargs
  # )
  return callbacks


class ModelCheckpoint(tf.keras.callbacks.ModelCheckpoint):
    def __init__(self, validation_dataset, num_validation_samples, *args, **kwargs):
        super(ModelCheckpoint, self).__init__(*args, **kwargs)
        self.validation_dataset = validation_dataset
        labels_dataset = self.validation_dataset.map(lambda image, label: label).unbatch()
        NUM_VALIDATION_IMAGES = num_validation_samples
        self.y_true = next(iter(labels_dataset.batch(NUM_VALIDATION_IMAGES))).numpy()
        
    def _save_model(self, epoch, logs):
        y_pred = self.model.predict(self.validation_dataset, verbose=0)
        current = roc_auc_score(self.y_true, y_pred)
        # print("ROC-AUC - epoch: {:d} - score: {:.6f}\n".format(epoch+1, score))

        if isinstance(self.save_freq,
                      int) or self.epochs_since_last_save >= self.period:
          self.epochs_since_last_save = 0
          filepath = self._get_file_path(epoch, logs)

          try:
            if self.save_best_only:
              if current is None:
                logging.warning('Can save best model only with %s available, '
                                'skipping.', self.monitor)
              else:
                if self.monitor_op(current, self.best):
                  if self.verbose > 0:
                    print('\nEpoch %05d: %s improved from %0.5f to %0.5f,'
                          ' saving model to %s' % (epoch + 1, self.monitor,
                                                  self.best, current, filepath))
                  self.best = current
                  if self.save_weights_only:
                    self.model.save_weights(filepath, overwrite=True)
                  else:
                    self.model.save(filepath, overwrite=True)
                else:
                  if self.verbose > 0:
                    print('\nEpoch %05d: %s did not improve from %0.5f (val_auc = %0.5f)' %
                          (epoch + 1, self.monitor, self.best, current))
            else:
              if self.verbose > 0:
                print('\nEpoch %05d: saving model to %s' % (epoch + 1, filepath))
              if self.save_weights_only:
                self.model.save_weights(filepath, overwrite=True)
              else:
                self.model.save(filepath, overwrite=True)

            self._maybe_remove_file()
          except IOError as e:
            # `e.errno` appears to be `None` so checking the content of `e.args[0]`.
            if 'is a directory' in six.ensure_str(e.args[0]):
              raise IOError('Please specify a non-directory filepath for '
                            'ModelCheckpoint. Filepath used is an existing '
                            'directory: {}'.format(filepath))
              
callbacks.get_callbacks = get_callbacks

In [0]:
#@title Classifier Trainer
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

import pprint
from typing import Any, Tuple, Text, Optional, Mapping

from absl import app
from absl import flags
from absl import logging
import tensorflow as tf
import tensorflow_addons as tfa
from focal_loss import BinaryFocalLoss

from official.vision.image_classification.classifier_trainer import *
from official.vision.image_classification.classifier_trainer import (
    _get_params_from_flags, _get_dataset_builders
)

from official.vision.image_classification import classifier_trainer

import tensorflow.keras.backend as K


class AUC(tf.keras.metrics.AUC):
  def update_state(self, y_true, y_pred, sample_weight=None):
    y_pred = K.sigmoid(y_pred)
    return super(AUC, self).update_state(y_true, y_pred, sample_weight)



def _get_metrics(one_hot: bool) -> Mapping[Text, Any]:
  """Get a dict of available metrics to track."""
  if one_hot:
    return {
        'auc': tf.keras.metrics.AUC(name='auc'),
        # 'auc': AUC(name='auc'),
    }
  else:
    return {
        'auc': tf.keras.metrics.AUC(name='auc'),
        # 'auc': AUC(name='auc'),
    }

def train_and_eval(
    params: base_configs.ExperimentConfig,
    strategy_override: tf.distribute.Strategy) -> Mapping[str, Any]:
  """Runs the train and eval path using compile/fit."""
  logging.info('Running train and eval.')

  # Note: for TPUs, strategy and scope should be created before the dataset
  strategy = strategy_override or distribution_utils.get_distribution_strategy(
      distribution_strategy=params.runtime.distribution_strategy,
      all_reduce_alg=params.runtime.all_reduce_alg,
      num_gpus=params.runtime.num_gpus,
      tpu_address=params.runtime.tpu)

  strategy_scope = distribution_utils.get_strategy_scope(strategy)

  logging.info('Detected %d devices.',
               strategy.num_replicas_in_sync if strategy else 1)

  label_smoothing = params.model.loss.label_smoothing
  one_hot = label_smoothing and label_smoothing > 0

  builders = _get_dataset_builders(params, strategy, one_hot)
  datasets = [builder.build() if builder else None for builder in builders]

  # Unpack datasets and builders based on train/val/test splits
  train_builder, validation_builder = builders  # pylint: disable=unbalanced-tuple-unpacking
  train_dataset, validation_dataset = datasets

  train_epochs = params.train.epochs
  train_steps = params.train.steps or train_builder.num_steps
  validation_steps = params.evaluation.steps or validation_builder.num_steps

  initialize(params, train_builder)

  logging.info('Global batch size: %d', train_builder.global_batch_size)

  with strategy_scope:
    model_params = params.model.model_params.as_dict()
    model = get_models()[params.model.name](**model_params)
    learning_rate = optimizer_factory.build_learning_rate(
        params=params.model.learning_rate,
        batch_size=train_builder.global_batch_size,
        train_steps=train_steps)
    optimizer = optimizer_factory.build_optimizer(
        optimizer_name=params.model.optimizer.name,
        base_learning_rate=learning_rate,
        params=params.model.optimizer.as_dict())

    metrics_map = _get_metrics(one_hot)
    metrics = [metrics_map[metric] for metric in params.train.metrics]

    # if one_hot:
    #   loss_obj = losses.CategoricalCrossentropy(
    #       label_smoothing=params.model.loss.label_smoothing)
    # else:
    #   loss_obj = losses.SparseCategoricalCrossentropy()

    # if one_hot:
    #   loss_obj = losses.BinaryCrossentropy(
    #       label_smoothing=params.model.loss.label_smoothing)
    # else:
    #   loss_obj = losses.BinaryCrossentropy()

    loss_obj = BinaryFocalLoss(
          pos_weight=1,
          gamma=2,
          label_smoothing=params.model.loss.label_smoothing
    )

    # loss_obj = tfr.keras.losses.get(
    #     tfr.losses.RankingLossKey.SIGMOID_CROSS_ENTROPY_LOSS)

    # loss_obj = losses.SurrogateLoss()
    # loss_obj = losses.RankBoostLoss()
    # loss_obj = losses.ROCAUCLoss()

    model.compile(optimizer=optimizer,
                  loss=loss_obj,
                  metrics=metrics)
    
    initial_epoch = 0
    if params.train.resume_checkpoint:
      initial_epoch = resume_from_checkpoint(model=model,
                                             model_dir=params.model_dir,
                                             train_steps=train_steps)

  serialize_config(params=params, model_dir=params.model_dir)

  callbacks = custom_callbacks.get_callbacks(
      validation_dataset,
      params.validation_dataset.num_examples,
      model_checkpoint=params.train.callbacks.enable_checkpoint_and_export,
      include_tensorboard=params.train.callbacks.enable_tensorboard,
      time_history=params.train.callbacks.enable_time_history,
      track_lr=params.train.tensorboard.track_lr,
      write_model_weights=params.train.tensorboard.write_model_weights,
      initial_step=initial_epoch * train_steps,
      batch_size=train_builder.global_batch_size,
      log_steps=params.train.time_history.log_steps,
      model_dir=params.model_dir)

  if params.evaluation.skip_eval:
    validation_kwargs = {}
  else:
    validation_kwargs = {
        'validation_data': validation_dataset,
        'validation_steps': validation_steps,
        'validation_freq': params.evaluation.epochs_between_evals,
    }

  history = model.fit(
      train_dataset,
      epochs=train_epochs,
      steps_per_epoch=train_steps,
      initial_epoch=initial_epoch,
      callbacks=callbacks,
      **validation_kwargs)

  validation_output = None
  if not params.evaluation.skip_eval:
    validation_output = model.evaluate(
        validation_dataset, steps=validation_steps, verbose=2)

  stats = common.build_stats(history,
                             validation_output,
                             callbacks)
  return stats

classifier_trainer.train_and_eval = train_and_eval
classifier_trainer._get_metrics = _get_metrics

In [22]:
%%writefile config.yaml

# Training configuration for EfficientNet 
runtime:
  model_dir: null
  mode: 'train_and_eval'
  distribution_strategy: 'tpu'
  run_eagerly: False
  enable_xla: True
train_dataset:
  name: 'imagenet2012'
  data_dir: null
  builder: 'records'
  split: 'train'
  one_hot: False
  num_classes: 2
  # num_examples: 26488
  num_examples: 46648
  image_size: 512
  batch_size: 32
  use_per_replica_batch_size: True
  dtype: 'bfloat16'
  augmenter:
    name: 'autoaugment'
validation_dataset:
  name: 'imagenet2012'
  data_dir: null
  builder: 'records'
  split: 'validation'
  cache: True
  one_hot: False
  num_classes: 2
  # num_examples: 6638
  num_examples: 11809
  image_size: 512
  batch_size: 64
  use_per_replica_batch_size: True
  dtype: 'bfloat16'
model:
  model_params:
    model_name: 'efficientnet-b3'
    model_weights_path: 'gs://recursion-kaggle/melanoma/efficientnet_b3_feature-vector'
    overrides:
      num_classes: 2
      batch_norm: 'tpu'
      dtype: 'bfloat16'
      # dropout_rate: 0.0
  optimizer:
    # RMSProp
    # name: 'rmsprop'
    # momentum: 0.9
    # decay: 0.9

    # AdamW
    name: 'adam'
    beta_1: 0.9
    beta_2: 0.999
    epsilon: 0.0000001
    moving_average_decay: 0.0
    # decay: 0.9
    # lookahead: True

  learning_rate:
    # boundaries: None
    # decay_epochs: 2.4
    # decay_rate: 0.8
    initial_lr: 0.001
    name: 'exponential'
    # scale_by_batch_size: 0.0078125
    warmup_epochs: 5
  loss:
    label_smoothing: 0.0
  num_classes: 2
train:
  resume_checkpoint: False
  epochs: 50
  metrics: ['auc']
  callbacks:
    enable_checkpoint_and_export: True
    enable_tensorboard: False
evaluation:
  epochs_between_evals: 1

Overwriting config.yaml


In [0]:
from absl import app
from absl import flags

In [0]:
define_classifier_flags()

In [0]:
#@title Train
logging.set_verbosity(logging.INFO)

if __name__ == '__main__':
  if '-f' in sys.argv:
    sys.argv.remove('-f')
  flags.FLAGS.mode = 'train_and_eval' 
  flags.FLAGS.model_type = 'efficientnet' 
  flags.FLAGS.dataset = 'imagenet' 
  flags.FLAGS.tpu = TPU_NAME 
  flags.FLAGS.model_dir = 'gs://recursion-kaggle/melanoma/models/model_b3_test' #@param {type:"string"}
  flags.FLAGS.data_dir = 'gs://recursion-kaggle/melanoma/stratified_ex/fold0' #@param {type:"string"}
  flags.FLAGS.config_file = 'config.yaml' #@param {type:"string"}

  app.run(main)

I0610 22:49:28.845302 140176018913152 classifier_trainer.py:185] Base params: {'evaluation': {'epochs_between_evals': 1, 'skip_eval': False, 'steps': None},
 'export': {'checkpoint': None, 'destination': None},
 'mode': None,
 'model': {'learning_rate': {'boundaries': None,
                             'decay_epochs': 2.4,
                             'decay_rate': 0.97,
                             'examples_per_epoch': None,
                             'initial_lr': 0.008,
                             'multipliers': None,
                             'name': 'exponential',
                             'scale_by_batch_size': 0.0078125,
                             'warmup_epochs': 5},
           'loss': {'label_smoothing': 0.1,
                    'loss_scale': None,
                    'name': 'categorical_crossentropy'},
           'model_params': {'copy_to_local': False,
                            'model_name': 'efficientnet-b0',
                            'model_weights_path': 

W0610 22:49:28.869886 140176018913152 tpu_strategy_util.py:70] TPU system grpc://10.49.17.50:8470 has already been initialized. Reinitializing the TPU can cause previously created variables on TPU to be lost.


INFO:tensorflow:Initializing the TPU system: grpc://10.49.17.50:8470


I0610 22:49:28.871539 140176018913152 tpu_strategy_util.py:72] Initializing the TPU system: grpc://10.49.17.50:8470


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


I0610 22:49:43.333533 140176018913152 tpu_system_metadata.py:146] *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


I0610 22:49:43.334408 140176018913152 tpu_system_metadata.py:146] *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
I0610 22:49:43.335729 140176018913152 <ipython-input-21-fbb1333ce392>:68] Detected 8 devices.
W0610 22:49:43.336300 140176018913152 classifier_trainer.py:109] label_smoothing not applied, so datasets will not be one hot encoded.
I0610 22:49:43.337088 140176018913152 dataset_factory.py:174] Using augmentation: autoaugment
I0610 22:49:43.337967 140176018913152 dataset_factory.py:174] Using augmentation: None
I0610 22:49:43.338646 140176018913152 <ipython-input-17-254e7a04ba02>:20] Using TFRecords to load data.
I0610 22:49:47.609058 140176018913152 <ipython-input-17-254e7a04ba02>:20] Using TFRecords to load data.
I0610 22:49:47.686149 140176018913152 dataset_builder.py:202] Load pre-computed datasetinfo (eg: splits) from bucket.
I0610 22:49:47.689069 140176018913152 dataset_info.py:431] Loading info from GCS for imagenet201

Epoch 1/50
182/182 [==============================] - ETA: 0s - auc: 0.6953 - loss: 0.6525
Epoch 00001: val_auc improved from -inf to 0.85723, saving model to gs://recursion-kaggle/melanoma/models/model_b3_test/model.ckpt
182/182 [==============================] - 206s 1s/step - auc: 0.6953 - loss: 0.6525 - val_auc: 0.8564 - val_loss: 0.2831
Epoch 2/50
 58/182 [========>.....................] - ETA: 2:50 - auc: 0.7650 - loss: 0.3587